In [1]:
import sys
sys.path.insert(0, '/Users/carsoncook/Dev/CS445/Group_Project_cs445')
import extractFeatures as ef

import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.recurrent import SimpleRNN, LSTM, GRU 
from keras.optimizers import RMSprop

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sn

#Constants
FEATURES = 135
# INPUT_SHAPE = (FEATURES, 1)
ROWS = 9
COLS = 15
N_CLASSES = 10

# Hyper Parameters
UNITS = 64
# L1_UNITS = 32
# L2_UNITS = 32
BATCH_SIZE = 128
EPOCHS = 100
DROPOUT = (.25, .5)

np.random.seed(10)

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


LSTM RNN

In [ ]:
# RNN LSTM
def build_and_run_model(X_train, X_test, Y_train, Y_test, units, rows, cols, batch_size, epochs):
    X_train = X_train.reshape(-1, rows, cols)
    X_test = X_test.reshape(-1, rows, cols)
    
    model = Sequential()
    model.add(LSTM(units, input_shape=(rows, cols),
                   return_sequences=True,
                   activation='tanh',
                   recurrent_activation ='sigmoid',
                   unit_forget_bias=True,
                  ))
#     model.add(LSTM(units, input_shape=(rows, cols),
#                    return_sequences=True,
#                    activation='tanh',
#                    recurrent_activation ='sigmoid',
#                    unit_forget_bias=True,
#                   ))
    model.add(LSTM(units, input_shape=(rows, cols),
                   activation='tanh',
                   recurrent_activation ='sigmoid',
                   unit_forget_bias=True,
                  ))
    model.add(Dense(N_CLASSES, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.fit(X_train, Y_train,
             batch_size=batch_size,
             epochs=epochs,
             verbose=1,
             validation_data=(X_test, Y_test))
    score = model.evaluate(X_test, Y_test, verbose=0)
    print("Score: ", score)
    return score[1]

Assess model using cross validation

In [ ]:
accs = np.zeros(shape=(10,), dtype='float64')
for i in range(10):
    (aud_X_train, aud_X_test, aud_y_train, aud_y_test,
    im_X_train, im_X_test, im_y_train, im_y_test,
    aud_X_train_norm, aud_X_test_norm, aud_y_train_norm, aud_y_test_norm,
    im_X_train_norm, im_X_test_norm, im_y_train_norm, im_y_test_norm) = ef.crossValidationIteration(i)  
    
    # One Hot Encoding
    aud_Y_train = keras.utils.to_categorical(aud_y_train, N_CLASSES)
    aud_Y_test = keras.utils.to_categorical(aud_y_test, N_CLASSES)
    
    accs[i] = build_and_run_model(aud_X_train, aud_X_test, aud_Y_train, aud_Y_test,
                                  UNITS, ROWS, COLS, BATCH_SIZE, EPOCHS)

Train on 7859 samples, validate on 873 samples
Epoch 1/100
7859/7859 [==============================] - 3s 332us/step - loss: 1.6977 - acc: 0.4414 - val_loss: 1.4429 - val_acc: 0.4845
Epoch 2/100
7859/7859 [==============================] - 1s 185us/step - loss: 1.0435 - acc: 0.6729 - val_loss: 1.3257 - val_acc: 0.5487
Epoch 3/100
7859/7859 [==============================] - 1s 178us/step - loss: 0.8062 - acc: 0.7464 - val_loss: 1.5284 - val_acc: 0.5052
Epoch 4/100
7859/7859 [==============================] - 1s 184us/step - loss: 0.6644 - acc: 0.7881 - val_loss: 1.7823 - val_acc: 0.4364
Epoch 5/100
7859/7859 [==============================] - 1s 184us/step - loss: 0.5694 - acc: 0.8206 - val_loss: 1.6850 - val_acc: 0.5074
Epoch 6/100
7859/7859 [==============================] - 1s 179us/step - loss: 0.5023 - acc: 0.8387 - val_loss: 1.7585 - val_acc: 0.4696
Epoch 7/100
7859/7859 [==============================] - 1s 184us/step - loss: 0.4437 - acc: 0.8585 - val_loss: 1.8284 - val_acc: 0